<a href="https://colab.research.google.com/github/jjunga1203/Books/blob/master/chat_Kobert_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

In [4]:
# ★
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-6fl8ewvh/kobert-tokenizer_3b157ad9abd2450bbc5dc9f5c3c19201
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-6fl8ewvh/kobert-tokenizer_3b157ad9abd2450bbc5dc9f5c3c19201
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=47692264c931d920c1b1976ff2c63eee1d423fea19459f175282070c0a2b1451
  Stored in directory: /tmp/pip-ephem-wheel-cache-mdih_d9c/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [2]:
!pip install gluonnlp==0.8.0

In [1]:
!pip install numpy==1.21.3

In [6]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import numpy as np
from tqdm import tqdm, tqdm_notebook

In [7]:
# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [9]:
# GPU 사용 시
device = torch.device("cuda:0")

In [39]:
# ★
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
# vocab = vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

# 어휘 관련 기능을 사용하여 vocab을 생성합니다.
vocab = tokenizer.get_vocab()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [17]:
import pandas as pd
df = pd.read_excel('./감성대화말뭉치(최종데이터)_Training.xlsx', index_col=0)


In [18]:
chatbot_df = df[['감정_대분류','사람문장1', '사람문장2', '사람문장3']]
chatbot_df.columns = ['Emotion', 'Sentence1', 'Sentence2', 'Sentence3']

In [19]:
emot = {'불안':0, '분노':1, '상처':2, '슬픔':3, '당황':4, '기쁨':5}
chatbot_df['Emotion'] = chatbot_df['Emotion'].map(emot)

<ipython-input-19-4f99096f4c28>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chatbot_df['Emotion'] = chatbot_df['Emotion'].map(emot)


In [20]:
data_list = []
for q1, q2, q3, label in zip(chatbot_df['Sentence1'], chatbot_df['Sentence2'], chatbot_df['Sentence3'], chatbot_df['Emotion'])  :
    if q1 != 'NaN':
        data = []
        data.append(q1)
        data.append(label)
        data_list.append(data)
    if q2 != 'NaN':
        data = []
        data.append(q2)
        data.append(label)
        data_list.append(data)
    if q3 != 'NaN':
        data = []
        data.append(q3)
        data.append(label)
        data_list.append(data)

In [21]:
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

print(len(dataset_train)) # 59604 출력
print(len(dataset_test)) # 19869 출력

116167
38723


In [22]:
from torch.utils.data import Dataset, DataLoader

In [29]:
## Setting parameters
max_len = 64
batch_size = 1
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [41]:
#tokenize
# tokenizer = get_tokenizer()
# from gluonnlp.data import BERTSPTokenizer
from torch.utils.data import DataLoader
from transformers import BertTokenizer
import torch

# 문장을 토크나이징하는 함수
def tokenize_sentences(sentences, tokenizer, max_len):
    # 토크나이저를 사용하여 문장을 토큰화하고 인코딩합니다.
    encodings = tokenizer(sentences, truncation=True, padding=True, max_length=max_len)
    return encodings

class BERTDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, sent_idx, label_idx, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.sentences = [i[sent_idx] for i in dataset]
        self.labels = [i[label_idx] for i in dataset]

    def __getitem__(self, idx):
        # 문장을 토큰화하고 인코딩합니다.
        encoding = tokenize_sentences(self.sentences[idx], self.tokenizer, self.max_len)
        label = torch.tensor(self.labels[idx])
        return encoding, label

    def __len__(self):
        return len(self.labels)

# BERT 토크나이저를 가져옵니다.
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# BERTDataset을 생성합니다.
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, max_len)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, max_len)

# DataLoader를 생성합니다.
train_dataloader = DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = DataLoader(data_test, batch_size=batch_size, num_workers=5)


In [35]:
train_dataloader

In [42]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [43]:
#정의한 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
# train_dataloader

In [45]:
import torch
from tqdm import tqdm

# 모델을 학습 모드로 설정합니다.
model.train()

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    train_total = 0
    test_total = 0



    # 훈련 데이터셋에 대한 반복문
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(train_dataloader):
        optimizer.zero_grad()

        # GPU로 데이터를 전송합니다.
        token_ids = token_ids.to(device)
        segment_ids = segment_ids.to(device)
        valid_length = valid_length.to(device)
        label = label.to(device)

        # 모델에 입력을 전달하여 예측값을 얻습니다.
        out = model(token_ids, attention_mask=valid_length, token_type_ids=segment_ids)

        # 손실을 계산합니다.
        loss = loss_fn(out.logits, label)

        # 역전파를 수행하여 그래디언트를 계산합니다.
        loss.backward()

        # 그래디언트 클리핑을 수행합니다.
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        # 가중치를 업데이트합니다.
        optimizer.step()
        scheduler.step()  # Learning rate schedule 업데이트

        # 정확도를 계산합니다.
        predicted_labels = torch.argmax(out.logits, dim=1)
        train_acc += torch.sum(predicted_labels == label).item()
        train_total += label.size(0)

        # 일정 간격마다 결과를 출력합니다.
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.item(), train_acc / train_total))

    # 훈련 정확도를 출력합니다.
    print("epoch {} train acc {}".format(e+1, train_acc / train_total))

    # 평가 데이터셋에 대한 반복문
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.to(device)
        segment_ids = segment_ids.to(device)
        valid_length = valid_length.to(device)
        label = label.to(device)

        with torch.no_grad():
            out = model(token_ids, attention_mask=valid_length, token_type_ids=segment_ids)
            predicted_labels = torch.argmax(out.logits, dim=1)
            test_acc += torch.sum(predicted_labels == label).item()
            test_total += label.size(0)

    # 테스트 정확도를 출력합니다.
    print("epoch {} test acc {}".format(e+1, test_acc / test_total))


ValueError: not enough values to unpack (expected 4, got 2)

In [46]:
# for batch_idx, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
#     print("Batch index:", batch_idx)
#     print("Token IDs:", token_ids)
#     print("Valid length:", valid_length)
#     print("Segment IDs:", segment_ids)
#     print("Label:", label)

ValueError: not enough values to unpack (expected 4, got 2)